In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8m.pt")

In [ ]:
cd /Users/emilygemmell/miniconda3/minicoco/data/all_images

In [ ]:
#make an empty csv file to store the extracted results data output
with open('/Users/emilygemmell/miniconda3/minicoco/results/pred_allother.csv', 'w') as creating_new_csv_file: 
       pass 

We ran the code below separately for classes 0 (persons), 2 (cars) and all the remaining classes together (1,3,5,6,7,9,10,11,12,13,14,15,16,32,33,34,35,36,38,56,57,58).

We used the following confidence and iou thresholds for each class after testing to check sensitivity and specificity of predictions.

Class 0 - persons: conf= 0.55, iou = 0.90

Class 2 - cars: conf = 0.50, iou = 0.95

All other classes (1,3,5,6,7,9,10,11,12,13,14,15,16,32,33,34,35,36,38,56,57,58): conf = 0.60, iou = 0.60


In [ ]:
import os
from os import listdir
import numpy as np
import cv2
import pandas as pd
import re


# create path to folder with images
folder_dir = "/Users/emilygemmell/miniconda3/minicoco/data/all_images/"

# run model, escaping any images that are not found
for i in range(0, len(os.listdir(folder_dir))):
    try:
        # inference
        result = model.predict(os.listdir(folder_dir)[i], 
                conf = 0.60, 
                iou = 0.60, 
                classes = [1,3,5,6,7,9,10,11,12,13,14,15,16,32,33,34,35,36,38,56,57,58],
                max_det = 300,
                agnostic_nms = False)
    except FileNotFoundError as e:
        print(f"FileNotFoundError successfully handled\n"
                    f"{e}")
        continue

            
        # make an empty list to store extracted data
    datalist = []

        # get image name
    img_name = result[0].path
            
        # get the total number of detected objects in the image
    detection_count = result[0].boxes.shape[0]
        
        # go through each detected object and extract the class number, class name and confidence
        # save these values with the image name in a list called "row"
        # save each row to the results folder
    for ii in range(detection_count):
        cls = int(result[0].boxes.cls[ii].item())
        name = result[0].names[cls]
        confidence = float(result[0].boxes.conf[ii].item())
        row = [img_name, cls, name, confidence]
        datalist.append(row)
        df = pd.DataFrame(datalist, columns = ['image_name', 'class_num', 'class', 'confidence'])
        df.to_csv('/Users/emilygemmell/miniconda3/minicoco/results/pred_allother.csv', mode='a', index=False, header=False)
    

Now run model for each class, setting the confidence threshold and iou threshold after a few trials to check accuracy. You will have to find a balance between sensitivity and specificity.

In [ ]:
allres = pd.read_csv('/Users/emilygemmell/miniconda3/minicoco/results/pred_allother.csv', header=None)

In [ ]:
allres